In [1]:
import tensorflow as tf
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import itertools
from sklearn.metrics import accuracy_score
print(tf.__version__)
import os

2.4.1


In [2]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Dropout, MaxPool2D, Dense, Conv2D, Flatten, BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm
from skimage.transform import resize

In [3]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7801351289072907850
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3044750132
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7558917462786453108
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [4]:
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size
def convert_bytes(size, unit=None):
    if unit == "KB":
        return str(round(size / 1024, 3))
    elif unit == "MB":
        return str(round(size / (1024 * 1024), 3)) 
    else:
        return print('File size: ' + str(size) + ' bytes')

In [5]:
device=tf.test.gpu_device_name()
print(device)

/device:GPU:0


In [6]:
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [7]:
mobilenet_img_size=(224,224,3)

In [8]:
x_test.shape

(10000, 32, 32, 3)

In [9]:
x_train=x_train[:500]
y_train=y_train[:500]
x_test=x_test[:500]
y_test=y_test[:500]

In [10]:
y_train = y_train.flatten()
y_test = y_test.flatten()

In [11]:
y_train

array([6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, 9, 9, 3, 2, 6, 4, 3,
       6, 6, 2, 6, 3, 5, 4, 0, 0, 9, 1, 3, 4, 0, 3, 7, 3, 3, 5, 2, 2, 7,
       1, 1, 1, 2, 2, 0, 9, 5, 7, 9, 2, 2, 5, 2, 4, 3, 1, 1, 8, 2, 1, 1,
       4, 9, 7, 8, 5, 9, 6, 7, 3, 1, 9, 0, 3, 1, 3, 5, 4, 5, 7, 7, 4, 7,
       9, 4, 2, 3, 8, 0, 1, 6, 1, 1, 4, 1, 8, 3, 9, 6, 6, 1, 8, 5, 2, 9,
       9, 8, 1, 7, 7, 0, 0, 6, 9, 1, 2, 2, 9, 2, 6, 6, 1, 9, 5, 0, 4, 7,
       6, 7, 1, 8, 1, 1, 2, 8, 1, 3, 3, 6, 2, 4, 9, 9, 5, 4, 3, 6, 7, 4,
       6, 8, 5, 5, 4, 3, 1, 8, 4, 7, 6, 0, 9, 5, 1, 3, 8, 2, 7, 5, 3, 4,
       1, 5, 7, 0, 4, 7, 5, 5, 1, 0, 9, 6, 9, 0, 8, 7, 8, 8, 2, 5, 2, 3,
       5, 0, 6, 1, 9, 3, 6, 9, 1, 3, 9, 6, 6, 7, 1, 0, 9, 5, 8, 5, 2, 9,
       0, 8, 8, 0, 6, 9, 1, 1, 6, 3, 7, 6, 6, 0, 6, 6, 1, 7, 1, 5, 8, 3,
       6, 6, 8, 6, 8, 4, 6, 6, 1, 3, 8, 3, 4, 1, 7, 1, 3, 8, 5, 1, 1, 4,
       0, 9, 3, 7, 4, 9, 9, 2, 4, 9, 9, 1, 0, 5, 9, 0, 8, 2, 1, 2, 0, 5,
       6, 3, 2, 7, 8, 8, 6, 0, 7, 9, 4, 5, 6, 4, 2,

In [12]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']



In [13]:
np.isnan(x_train).any()

False

In [14]:
np.isnan(x_train).any()

False

In [15]:
def resize_images(images,img_size):
    resized_images = []
    
    for i in tqdm(range(len(images))):
        resized_images.append(resize(images[i],(img_size),mode = 'constant'))
    
    return np.array(resized_images,dtype='float32')

In [16]:
print('Loading train data for MobileNet :')
x_train = resize_images(x_train,mobilenet_img_size)

print('Loading test data for MobileNet :')
x_test = resize_images(x_test,mobilenet_img_size)

Loading train data for MobileNet :


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:05<00:00, 92.85it/s]


Loading test data for MobileNet :


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:05<00:00, 92.41it/s]


In [17]:
input_shape = (224,224,3)

x_train=x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 3)
x_train=x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 3)
x_test=x_test / 255.0

In [18]:
y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

In [19]:
y_train[0]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)>

In [20]:
batch_size = 16
num_classes = 10
epochs = 5

In [21]:
mobilenet_img_size=(224,224,3)

In [22]:
base_model = MobileNet(include_top=False, weights='imagenet',input_shape = mobilenet_img_size, classes=10)
base_model.trainable = True 

for layer in base_model.layers[:50]:
    layer.trainable =  False

MobileNet_model = tf.keras.models.Sequential()
MobileNet_model.add(base_model)
MobileNet_model.add(Flatten())
MobileNet_model.add(Dense(10,activation=('softmax')))

early_stopping = EarlyStopping(min_delta = 0.001,patience = 20,restore_best_weights = True,verbose = 0)

# Compile
MobileNet_model.compile(optimizer = "adam" , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

# Train
Mobile = MobileNet_model.fit(x_train, y_train, batch_size = 16, epochs = 5,callbacks = [early_stopping])

MobileNet_model.summary()

Epoch 1/5
32/32 [==============================] - 8s 117ms/step - loss: 29.4245 - accuracy: 0.1482
Epoch 2/5
32/32 [==============================] - 3s 100ms/step - loss: 6.9887 - accuracy: 0.1289
Epoch 3/5
32/32 [==============================] - 3s 101ms/step - loss: 2.8582 - accuracy: 0.0993
Epoch 4/5
32/32 [==============================] - 3s 100ms/step - loss: 2.6098 - accuracy: 0.0763
Epoch 5/5
32/32 [==============================] - 3s 100ms/step - loss: 2.6039 - accuracy: 0.0899
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Function (None, 7, 7, 1024)        3228864   
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                501770    
Total params: 3,730,63

In [23]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(32, 3, padding='same', input_shape=x_train.shape[1:], activation='relu'),
#     tf.keras.layers.Conv2D(32, 3, activation='relu'),
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Dropout(0.25),

#     tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
#     tf.keras.layers.Conv2D(64, 3, activation='relu'),
    
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Dropout(0.25),

#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(num_classes, activation='softmax'),
# ])

# model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-06),
#             loss='categorical_crossentropy', metrics=['acc'])

In [24]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(32, 3, padding='same', input_shape=x_train.shape[1:], activation='relu'),
#     tf.keras.layers.Conv2D(32, 3, activation='relu'),
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Dropout(0.25),

#     tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
#     tf.keras.layers.Conv2D(64, 3, activation='relu'),
#     tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
#     tf.keras.layers.Conv2D(128, 3, activation='relu'),
    
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Dropout(0.25),

#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(num_classes, activation='softmax'),
# ])

# model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-06),
#             loss='categorical_crossentropy', metrics=['acc'])

In [25]:
# with tf.device('/gpu:0'):
#     history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

In [26]:
# fig, ax = plt.subplots(2,1)
# ax[0].plot(history.history['loss'], color='b', label="Training Loss")
# legend = ax[0].legend(loc='best', shadow=True)

# ax[1].plot(history.history['acc'], color='b', label="Training Accuracy")
# legend = ax[1].legend(loc='best', shadow=True)

In [27]:
# test_loss, test_acc = model.evaluate(x_test, y_test)

In [28]:
# # Predict the values from the validation dataset
# y_pred = model.predict(x_test)
# # Convert predictions classes to one hot vectors 
# y_pred_classes = np.argmax(y_pred,axis = 1) 
# # Convert validation observations to one hot vectors
# y_true = np.argmax(y_test,axis = 1)
# # compute the confusion matrix
# confusion_mtx = tf.math.confusion_matrix(y_true, y_pred_classes)

In [29]:
# plt.figure(figsize=(12, 9))
# c = sns.heatmap(confusion_mtx, annot=True, fmt='g')
# c.set(xticklabels=classes, yticklabels=classes)

In [30]:
model=MobileNet_model

In [31]:
model.save("./cifar_mobilenet.h5")

In [32]:
model.save("./cifar_mobilenet")

INFO:tensorflow:Assets written to: ./cifar_mobilenet\assets


In [33]:
model=tf.keras.models.load_model("./cifar_mobilenet.h5")
test_loss, test_acc = model.evaluate(x_test, y_test)

16/16 [==============================] - 2s 76ms/step - loss: 4.5934 - accuracy: 0.1020


In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Function (None, 7, 7, 1024)        3228864   
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                501770    
Total params: 3,730,634
Trainable params: 3,166,218
Non-trainable params: 564,416
_________________________________________________________________


In [35]:
test_acc

0.10199999809265137

In [36]:
converter=tf.lite.TFLiteConverter.from_saved_model("./cifar_mobilenet")
tflite_model=converter.convert()

In [37]:
converter=tf.lite.TFLiteConverter.from_saved_model("./cifar_mobilenet")
converter.optimizations=[tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model=converter.convert()

In [38]:
interpreter= tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()




In [39]:
interpreter.resize_tensor_input(input_details[0]['index'], (500, 224,224,3))
interpreter.resize_tensor_input(output_details[0]['index'], (500, 10))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [500 224 224   3]
Input Type: <class 'numpy.float32'>
Output Shape: [500  10]
Output Type: <class 'numpy.float32'>


In [40]:
test_imgs_numpy = np.array(x_test, dtype=np.float32)

In [41]:
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

Prediction results shape: (500, 10)


In [42]:
tflite_model_acc = accuracy_score(prediction_classes, np.argmax(y_test,axis=1))


In [43]:
interpreter= tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()
input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()

In [44]:
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [  1 224 224   3]
Input Type: <class 'numpy.float32'>
Output Shape: [ 1 10]
Output Type: <class 'numpy.float32'>


In [45]:
interpreter.resize_tensor_input(input_details[0]['index'], (500, 224,224,3))
interpreter.resize_tensor_input(output_details[0]['index'], (500, 10))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [500 224 224   3]
Input Type: <class 'numpy.float32'>
Output Shape: [500  10]
Output Type: <class 'numpy.float32'>


In [46]:
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

Prediction results shape: (500, 10)


In [47]:
tflite_model_quant_acc = accuracy_score(prediction_classes, np.argmax(y_test,axis=1))


In [48]:
keras_model_size = get_file_size("cifar_mobilenet.h5")

In [49]:
TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"
open(TF_LITE_MODEL_FILE_NAME, "wb").write(tflite_model)
keras_model_tflite_size=get_file_size("tf_lite_model.tflite")

In [50]:
TF_LITE_MODEL_FLOAT_16_FILE_NAME = "tf_lite_float_16_model.tflite"
keras_model_tflite_quant_size=open(TF_LITE_MODEL_FLOAT_16_FILE_NAME, "wb").write(tflite_quant_model)

In [51]:
print("tf model accuracy:",test_acc)
print("tflite model accuracy:",tflite_model_acc)
print("tflite quant model accuracy:",tflite_model_quant_acc)

tf model accuracy: 0.10199999809265137
tflite model accuracy: 0.102
tflite quant model accuracy: 0.102


In [52]:
print("Keras model size",convert_bytes(keras_model_size,"KB"))
print("Keras tflite model size",convert_bytes(keras_model_tflite_size,"KB"))
print("Keras tflite quant model size",convert_bytes(keras_model_tflite_quant_size,"KB"))


Keras model size 39552.023
Keras tflite model size 14455.184
Keras tflite quant model size 7239.281


In [53]:
print("model size",convert_bytes(keras_model_size,"MB"))
print("tflite model size",convert_bytes(keras_model_tflite_size,"MB"))
print("tflite quant model size",convert_bytes(keras_model_tflite_quant_size,"MB"))


model size 38.625
tflite model size 14.116
tflite quant model size 7.07


In [54]:
import tensorflow_model_optimization as tfmot

In [59]:
base_model = MobileNet(include_top=False, weights='imagenet',input_shape = mobilenet_img_size, classes=10)
base_model.trainable = True 
# base_model=tfmot.quantization.keras.quantize_model(base_model)
for layer in base_model.layers[:50]:
    layer.trainable =  False

MobileNet_model = tf.keras.models.Sequential()
MobileNet_model.add(base_model)
MobileNet_model.add(Flatten())
MobileNet_model.add(Dense(10,activation=('softmax')))

early_stopping = EarlyStopping(min_delta = 0.001,patience = 20,restore_best_weights = True,verbose = 0)



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Function (None, 7, 7, 1024)        3228864   
_________________________________________________________________
flatten_2 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                501770    
Total params: 3,730,634
Trainable params: 3,166,218
Non-trainable params: 564,416
_________________________________________________________________


In [62]:
MobileNet_model=tfmot.quantization.keras.quantize_model(MobileNet_model)
MobileNet_model.compile(optimizer = "adam" , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

# Train


MobileNet_model.summary()

ValueError: Quantizing a tf.keras Model inside another tf.keras Model is not supported.

In [57]:
# model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-06),
#             loss='categorical_crossentropy', metrics=['acc'])
# with tf.device('/gpu:0'):
#     history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)


In [58]:
Mobile = MobileNet_model.fit(x_train, y_train, batch_size = 16, epochs = 5,callbacks = [early_stopping])

Epoch 1/5


InvalidArgumentError:  The primary convolution algorithm failed memory allocation, while a secondary algorithm is not provided.
	 [[node gradient_tape/sequential_1/mobilenet_1.00_224/quant_conv_pw_13/Conv2D/Conv2DBackpropFilter (defined at <ipython-input-58-7476d99938a4>:1) ]] [Op:__inference_train_function_81259]

Errors may have originated from an input operation.
Input Source operations connected to node gradient_tape/sequential_1/mobilenet_1.00_224/quant_conv_pw_13/Conv2D/Conv2DBackpropFilter:
 sequential_1/mobilenet_1.00_224/quant_conv_dw_13_relu/MovingAvgQuantize/FakeQuantWithMinMaxVars (defined at C:\conda\envs\ml\lib\site-packages\tensorflow_model_optimization\python\core\quantization\keras\quant_ops.py:343)

Function call stack:
train_function


In [ ]:
model.save("cifar_aware.h5")

In [ ]:
model.save("./cifar_aware")

In [ ]:
test_loss_aware, test_acc_aware= model.evaluate(x_test, y_test)

In [ ]:
converter=tf.lite.TFLiteConverter.from_saved_model("cifar_aware")
tflite_aware_model=converter.convert()

In [ ]:
interpreter= tf.lite.Interpreter(model_content=tflite_aware_model)
interpreter.allocate_tensors()
input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()




In [ ]:
interpreter.resize_tensor_input(input_details[0]['index'], (10000, 32, 32,3))
interpreter.resize_tensor_input(output_details[0]['index'], (10000, 10))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

In [ ]:
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

In [ ]:
tflite_aware_model_acc = accuracy_score(prediction_classes, np.argmax(y_test,axis=1))


In [ ]:
tflite_aware_model_acc